In [1]:
# Load libraries

# # Plotting utils 
import matplotlib.pyplot as plt 
import matplotlib.colors as colors
import matplotlib.ticker as ticker 
import matplotlib.patches as patches
import matplotlib as matplotlib
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable

# import seaborn as sns

# Analysis
import time
import numpy as np 
import xarray as xr
import datetime
from   datetime import date, timedelta
import pandas as pd 
import pickle
import scipy.stats as stats
import glob
import os 

import metpy.calc as mpc
from metpy.units import units
from metpy.plots import Hodograph, SkewT

import warnings
warnings.filterwarnings('ignore')

## Import Ngl with pyn_env active 
import Ngl



**Useful functions**

In [2]:
# Grabbed from Brian M. to use time midpoints, not end periods
def cesm_correct_time(ds):
    """Given a Dataset, check for time_bnds,
       and use avg(time_bnds) to replace the time coordinate.
       Purpose is to center the timestamp on the averaging inverval.   
       NOTE: ds should have been loaded using `decode_times=False`
    """
    assert 'time_bnds' in ds
    assert 'time' in ds
    correct_time_values = ds['time_bnds'].mean(dim='nbnd')
    # copy any metadata:
    correct_time_values.attrs = ds['time'].attrs
    ds = ds.assign_coords({"time": correct_time_values})
    ds = xr.decode_cf(ds)  # decode to datetime objects
    return ds


In [3]:
def interpolateToPressure_v2(DS, varName, pressGoals):
#     nCases = len(DSin.case.values)
#     nTimes = len(DSin.time.values)
    
#     saveOut = np.full([nTimes,len(pressGoals),1,1], np.nan)

    ## For the larger arrays, need to operate case-by-case; input to vinth2p can only be 3 or 4 dimensions. 
#     for iCase in range(nCases): 
#     DS = DSin

    p0mb = DS.P0.values[0]/100        # mb
    
    # Pull out hya/hyb profiles 
    hyam = DS.hyam.values[0,:]
    hybm = DS.hybm.values[0,:]
    hyai = DS.hyai.values[0,:]
    hybi = DS.hybi.values[0,:]

    
#     tempFile = listFilesHMG_camh0[0]
#     dsTemp   = xr.open_dataset(tempFile, decode_times=True)

#     hyam = dsTemp.hyam
#     hybm = dsTemp.hybm
#     hyai = dsTemp.hyai
#     hybi = dsTemp.hybi
#     p0mb = dsTemp.P0.values/100.0 

    # Surface pressure with time dimension
    PS   = DS.PS.values              # Pa 

    # Converting variables: 
    if np.shape(DS[varName].values)[1]==len(DS.ilev.values):
        varInterp = Ngl.vinth2p(DS[varName].values,hyai,hybi,pressGoals,PS,1,p0mb,1,True)
    elif np.shape(DS[varName].values)[1]==len(DS.lev.values):
        varInterp = Ngl.vinth2p(DS[varName].values,hyam,hybm,pressGoals,PS,1,p0mb,1,True)

    saveOut = varInterp
    
    return saveOut


## Read in data

In [4]:
## Where files are saved + start of casename
# dataDir = '/glade/scratch/mdfowler/CLASP_ensOutput/'
dataDir = '/glade/scratch/mdfowler/CLASP_ensOutput/multiplier/mult10_2018/'

htgFileStart = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HTGens'
# hmgFileStart = 'FSCAM.T42_T42.CLASP_fullycoupled_FinalOption.onlyThlRt.HMGens'

## Extra vars I want, but they're on the h0 files instead of the h1 files (same time step, just different stream) 
budgetTerms = ['PS','P0','hybm','hyam','hyai','hybi','thlp2','thlp2_bt','thlp2_cl','thlp2_dp1','thlp2_dp2','thlp2_forcing','thlp2_ma','thlp2_mc','thlp2_pd',
                         'thlp2_sf','thlp2_ta','thlp2_tp','thlp2_zt', 
                          'wp2','wp2_ac','wp2_bp','wp2_bt','wp2_cl','wp2_dp1','wp2_dp2','wp2_ma', 'wp2_pd','wp2_pr1','wp2_pr2',
                          'wp2_pr3','wp2_sf','wp2_ta','wp2_zt',
                          'wp2rcp','wp2rtp','wp2thlp','wp2thvp','wp3_on_wp2','wp3_on_wp2_zt','wprtp',
                          'wprtp2','wprtp_ac','wprtp_bp','wprtp_bt','wprtp_cl','wprtp_dp1','wprtp_enter_mfl','wprtp_exit_mfl',
                          'wprtp_forcing','wprtp_ma','wprtp_mc','wprtp_mfl','wprtp_mfl_max','wprtp_mfl_min','wprtp_pd',
                          'wprtp_pr1','wprtp_pr2','wprtp_pr3','wprtp_sicl','wprtp_ta','wprtp_tp','wprtp_zt',
                          'wprtpthlp','wpthlp','wpthlp2','wpthlp_ac','wpthlp_bp','wpthlp_bt','wpthlp_cl','wpthlp_dp1','wpthlp_entermfl',
                          'wpthlp_exit_mfl','wpthlp_forcing','wpthlp_ma','wpthlp_mc','wpthlp_mfl','wpthlp_mfl_max','wpthlp_mfl_min',
                          'wpthlp_pr1','wpthlp_pr2','wpthlp_pr3','wpthlp_sicl','wpthlp_ta','wpthlp_tp','wpthlp_zt']

## Other settings to build up case names 
pertVals = np.asarray(['00'])


pertCount = 0
for iPert in range(len(pertVals)): 
    
    ## **** IMPORTANT: I've added the 2016 option to the listed files to look at just one year right now
        
    # listFilesHMG_camh0 = np.sort(glob.glob(dataDir+hmgFileStart+'*pert'+pertVals[iPert]+'*cam.h0*'))
    listFilesHTG_camh0 = np.sort(glob.glob(dataDir+htgFileStart+'*pert'+pertVals[iPert]+'*cam.h0*'))
    
    fileCount=0
    for iFile in range(len(listFilesHTG_camh0)):
        # with xr.open_dataset(listFilesHMG_camh0[iFile], decode_times=False) as hmgDS: 
        #     hmgDS         = cesm_correct_time(hmgDS)
        #     hmgDS         = hmgDS[budgetTerms]
        #     hmgDS['time'] = hmgDS.indexes['time'].to_datetimeindex() 
        with xr.open_dataset(listFilesHTG_camh0[iFile], decode_times=False) as htgDS: 
            htgDS         = cesm_correct_time(htgDS)
            htgDS         = htgDS[budgetTerms]
            htgDS['time'] = htgDS.indexes['time'].to_datetimeindex()
        
  
        iTimeStart_day2  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(1,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(2,'D'))))[0]
        iTimeStart_day3  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(2,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(3,'D'))))[0]
        iTimeStart_day4  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(3,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(4,'D'))))[0]
        iTimeStart_day5  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(4,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(5,'D'))))[0]
        iTimeStart_day6  = np.where( (htgDS.time.values >= (htgDS.time.values[0] + np.timedelta64(5,'D'))) & 
                                     (htgDS.time.values <= (htgDS.time.values[0] + np.timedelta64(6,'D'))))[0]
        

        # Build larger array 
        if fileCount==0:
            # hmg_allDay2      = hmgDS.isel(time=iTimeStart_day2)
            # hmg_allDay3      = hmgDS.isel(time=iTimeStart_day3)
            # hmg_allDay4      = hmgDS.isel(time=iTimeStart_day4)
            # hmg_allDay5      = hmgDS.isel(time=iTimeStart_day5)
            # hmg_allDay6      = hmgDS.isel(time=iTimeStart_day6)
            
            htg_allDay2      = htgDS.isel(time=iTimeStart_day2)
            htg_allDay3      = htgDS.isel(time=iTimeStart_day3)
            htg_allDay4      = htgDS.isel(time=iTimeStart_day4)
            htg_allDay5      = htgDS.isel(time=iTimeStart_day5)
            htg_allDay6      = htgDS.isel(time=iTimeStart_day6)
            
            
        else: 
            # hmg_allDay2      = xr.concat([hmg_allDay2,    hmgDS.isel(time=iTimeStart_day2)],  dim='time')
            # hmg_allDay3      = xr.concat([hmg_allDay3,    hmgDS.isel(time=iTimeStart_day3)],  dim='time')
            # hmg_allDay4      = xr.concat([hmg_allDay4,    hmgDS.isel(time=iTimeStart_day4)],  dim='time')
            # hmg_allDay5      = xr.concat([hmg_allDay5,    hmgDS.isel(time=iTimeStart_day5)],  dim='time')
            # hmg_allDay6      = xr.concat([hmg_allDay6,    hmgDS.isel(time=iTimeStart_day6)],  dim='time')
            
            htg_allDay2      = xr.concat([htg_allDay2,    htgDS.isel(time=iTimeStart_day2)],  dim='time')
            htg_allDay3      = xr.concat([htg_allDay3,    htgDS.isel(time=iTimeStart_day3)],  dim='time')
            htg_allDay4      = xr.concat([htg_allDay4,    htgDS.isel(time=iTimeStart_day4)],  dim='time')
            htg_allDay5      = xr.concat([htg_allDay5,    htgDS.isel(time=iTimeStart_day5)],  dim='time')
            htg_allDay6      = xr.concat([htg_allDay6,    htgDS.isel(time=iTimeStart_day6)],  dim='time')   
                        
        fileCount = fileCount+1
        print('Done with day %i of %i ' % (iFile, len(listFilesHTG_camh0)) )
            
    ## Combine into larger HTG or HMG arrays for all pertlim experiments 
    # hmg_allDay2  = hmg_allDay2.assign_coords({"pertlim": iPert})
    # hmg_allDay3  = hmg_allDay3.assign_coords({"pertlim": iPert})
    # hmg_allDay4  = hmg_allDay4.assign_coords({"pertlim": iPert})
    # hmg_allDay5  = hmg_allDay5.assign_coords({"pertlim": iPert})
    # hmg_allDay6  = hmg_allDay6.assign_coords({"pertlim": iPert})
    
    htg_allDay2  = htg_allDay2.assign_coords({"pertlim": iPert})
    htg_allDay3  = htg_allDay3.assign_coords({"pertlim": iPert})
    htg_allDay4  = htg_allDay4.assign_coords({"pertlim": iPert})
    htg_allDay5  = htg_allDay5.assign_coords({"pertlim": iPert})
    htg_allDay6  = htg_allDay6.assign_coords({"pertlim": iPert})

  

    if pertCount==0: 
        # hmgDS_pertDay2 = hmg_allDay2
        # hmgDS_pertDay3 = hmg_allDay3
        # hmgDS_pertDay4 = hmg_allDay4
        # hmgDS_pertDay5 = hmg_allDay5
        # hmgDS_pertDay6 = hmg_allDay6
        
        htgDS_pertDay2 = htg_allDay2
        htgDS_pertDay3 = htg_allDay3
        htgDS_pertDay4 = htg_allDay4
        htgDS_pertDay5 = htg_allDay5
        htgDS_pertDay6 = htg_allDay6
      
 
    else: 
#         hmgDS_pertDay2 = xr.concat([hmgDS_pertDay2, hmg_allDay2], "pertlim")
#         hmgDS_pertDay3 = xr.concat([hmgDS_pertDay3, hmg_allDay3], "pertlim")
#         hmgDS_pertDay4 = xr.concat([hmgDS_pertDay4, hmg_allDay4], "pertlim")
#         hmgDS_pertDay5 = xr.concat([hmgDS_pertDay5, hmg_allDay5], "pertlim")
#         hmgDS_pertDay6 = xr.concat([hmgDS_pertDay6, hmg_allDay6], "pertlim")
        
        htgDS_pertDay2 = xr.concat([htgDS_pertDay2, htg_allDay2], "pertlim")
        htgDS_pertDay3 = xr.concat([htgDS_pertDay3, htg_allDay3], "pertlim")
        htgDS_pertDay4 = xr.concat([htgDS_pertDay4, htg_allDay4], "pertlim")
        htgDS_pertDay5 = xr.concat([htgDS_pertDay5, htg_allDay5], "pertlim")
        htgDS_pertDay6 = xr.concat([htgDS_pertDay6, htg_allDay6], "pertlim")
    
    
    print('Done with pertlim ', pertCount)
    
    pertCount=pertCount+1
            

Done with day 0 of 89 
Done with day 1 of 89 
Done with day 2 of 89 
Done with day 3 of 89 
Done with day 4 of 89 
Done with day 5 of 89 
Done with day 6 of 89 
Done with day 7 of 89 
Done with day 8 of 89 
Done with day 9 of 89 
Done with day 10 of 89 
Done with day 11 of 89 
Done with day 12 of 89 
Done with day 13 of 89 
Done with day 14 of 89 
Done with day 15 of 89 
Done with day 16 of 89 
Done with day 17 of 89 
Done with day 18 of 89 
Done with day 19 of 89 
Done with day 20 of 89 
Done with day 21 of 89 
Done with day 22 of 89 
Done with day 23 of 89 
Done with day 24 of 89 
Done with day 25 of 89 
Done with day 26 of 89 
Done with day 27 of 89 
Done with day 28 of 89 
Done with day 29 of 89 
Done with day 30 of 89 
Done with day 31 of 89 
Done with day 32 of 89 
Done with day 33 of 89 
Done with day 34 of 89 
Done with day 35 of 89 
Done with day 36 of 89 
Done with day 37 of 89 
Done with day 38 of 89 
Done with day 39 of 89 
Done with day 40 of 89 
Done with day 41 of 89 
Do

In [5]:
## Convert to local times...
# - - - - - - - - - - - - - - 
# hmgDS_pertDay2_local     = hmgDS_pertDay2.copy(deep=True)
# hmgDS_pertDay3_local     = hmgDS_pertDay3.copy(deep=True)
# hmgDS_pertDay4_local     = hmgDS_pertDay4.copy(deep=True)
# hmgDS_pertDay5_local     = hmgDS_pertDay5.copy(deep=True)
# hmgDS_pertDay6_local     = hmgDS_pertDay6.copy(deep=True)

htgDS_pertDay2_local     = htgDS_pertDay2.copy(deep=True)
htgDS_pertDay3_local     = htgDS_pertDay3.copy(deep=True)
htgDS_pertDay4_local     = htgDS_pertDay4.copy(deep=True)
htgDS_pertDay5_local     = htgDS_pertDay5.copy(deep=True)
htgDS_pertDay6_local     = htgDS_pertDay6.copy(deep=True)



# Confirmed that all the times are identical, so using the same local time arrays
localTimes_day2 = htgDS_pertDay2['time'].values - np.timedelta64(5,'h')
localTimes_day3 = htgDS_pertDay3['time'].values - np.timedelta64(5,'h')
localTimes_day4 = htgDS_pertDay4['time'].values - np.timedelta64(5,'h')
localTimes_day5 = htgDS_pertDay5['time'].values - np.timedelta64(5,'h')
localTimes_day6 = htgDS_pertDay6['time'].values - np.timedelta64(5,'h')


# Replace time dimension with local time
# hmgDS_pertDay2_local     = hmgDS_pertDay2_local.assign_coords({"time": localTimes_day2})
# hmgDS_pertDay3_local     = hmgDS_pertDay3_local.assign_coords({"time": localTimes_day3})
# hmgDS_pertDay4_local     = hmgDS_pertDay4_local.assign_coords({"time": localTimes_day4})
# hmgDS_pertDay5_local     = hmgDS_pertDay5_local.assign_coords({"time": localTimes_day5})
# hmgDS_pertDay6_local     = hmgDS_pertDay6_local.assign_coords({"time": localTimes_day6})

htgDS_pertDay2_local     = htgDS_pertDay2_local.assign_coords({"time": localTimes_day2})
htgDS_pertDay3_local     = htgDS_pertDay3_local.assign_coords({"time": localTimes_day3})
htgDS_pertDay4_local     = htgDS_pertDay4_local.assign_coords({"time": localTimes_day4})
htgDS_pertDay5_local     = htgDS_pertDay5_local.assign_coords({"time": localTimes_day5})
htgDS_pertDay6_local     = htgDS_pertDay6_local.assign_coords({"time": localTimes_day6})

print('First four times in UTC:\n',   htgDS_pertDay2.time.values[0:5])
print('First four times in local:\n', htgDS_pertDay2_local.time.values[0:5])



First four times in UTC:
 ['2018-06-02T05:03:30.000000000' '2018-06-02T05:08:30.000000000'
 '2018-06-02T05:13:30.000000000' '2018-06-02T05:18:30.000000000'
 '2018-06-02T05:23:30.000000000']
First four times in local:
 ['2018-06-02T00:03:30.000000000' '2018-06-02T00:08:30.000000000'
 '2018-06-02T00:13:30.000000000' '2018-06-02T00:18:30.000000000'
 '2018-06-02T00:23:30.000000000']


## Process the data as we've done before

In [6]:
def process_camData(DS):
 
    
    ## Interpolate to standard levels 
    # - - - - - - - - - - - - - - - - - - - - - - - -
    print('Beginning interpolation...') 
    
    # Decide on levels to interpoalte to and add to larger arrays
    pnew64 = np.arange(200.0,980.0,10.0) 

    DS = DS.assign_coords({"levInterp": pnew64})

    varSels = np.asarray(['thlp2','thlp2_bt','thlp2_cl','thlp2_dp1','thlp2_dp2','thlp2_forcing','thlp2_ma','thlp2_mc','thlp2_pd',
                         'thlp2_sf','thlp2_ta','thlp2_tp','thlp2_zt', 
                          'wp2','wp2_ac','wp2_bp','wp2_bt','wp2_cl','wp2_dp1','wp2_dp2','wp2_ma', 'wp2_pd','wp2_pr1','wp2_pr2',
                          'wp2_pr3','wp2_sf','wp2_ta','wp2_zt',
                          'wp2rcp','wp2rtp','wp2thlp','wp2thvp','wp3_on_wp2','wp3_on_wp2_zt','wprtp',
                          'wprtp2','wprtp_ac','wprtp_bp','wprtp_bt','wprtp_cl','wprtp_dp1','wprtp_enter_mfl','wprtp_exit_mfl',
                          'wprtp_forcing','wprtp_ma','wprtp_mc','wprtp_mfl','wprtp_mfl_max','wprtp_mfl_min','wprtp_pd',
                          'wprtp_pr1','wprtp_pr2','wprtp_pr3','wprtp_sicl','wprtp_ta','wprtp_tp','wprtp_zt',
                          'wprtpthlp','wpthlp','wpthlp2','wpthlp_ac','wpthlp_bp','wpthlp_bt','wpthlp_cl','wpthlp_dp1','wpthlp_entermfl',
                          'wpthlp_exit_mfl','wpthlp_forcing','wpthlp_ma','wpthlp_mc','wpthlp_mfl','wpthlp_mfl_max','wpthlp_mfl_min',
                          'wpthlp_pr1','wpthlp_pr2','wpthlp_pr3','wpthlp_sicl','wpthlp_ta','wpthlp_tp','wpthlp_zt'])

    for iVar in range(len(varSels)): 
        varUnits = DS[varSels[iVar]].units
        varName  = DS[varSels[iVar]].long_name

        # Interpolate variables and add to larger arrays 
        interpVar_real = interpolateToPressure_v2(DS, varSels[iVar], pnew64)

        DS[varSels[iVar]+'_interp']  = (('time','levInterp','lat','lon'), interpVar_real)

        ## Assign attibutes 
        DS[varSels[iVar]+'_interp'].attrs['units']     = varUnits
        DS[varSels[iVar]+'_interp'].attrs['long_name'] = varName

    return DS



In [7]:
# ## Process data (HMG)
# hmgDS_local_processed_day2 = process_camData(hmgDS_pertDay2_local)
# hmgDS_local_processed_day3 = process_camData(hmgDS_pertDay3_local)
# hmgDS_local_processed_day4 = process_camData(hmgDS_pertDay4_local)
# hmgDS_local_processed_day5 = process_camData(hmgDS_pertDay5_local)
# hmgDS_local_processed_day6 = process_camData(hmgDS_pertDay6_local)
# print('Done processing HMG DS') 

htgDS_local_processed_day2 = process_camData(htgDS_pertDay2_local)
htgDS_local_processed_day3 = process_camData(htgDS_pertDay3_local)
htgDS_local_processed_day4 = process_camData(htgDS_pertDay4_local)
htgDS_local_processed_day5 = process_camData(htgDS_pertDay5_local)
htgDS_local_processed_day6 = process_camData(htgDS_pertDay6_local)
print('Done processing HTG DS') 



Beginning interpolation...
Beginning interpolation...
Beginning interpolation...
Beginning interpolation...
Beginning interpolation...
Done processing HTG DS


In [8]:
# del hmgDS_pertDay2_local
# del hmgDS_pertDay3_local
# del hmgDS_pertDay4_local
# del hmgDS_pertDay5_local
# del hmgDS_pertDay6_local

del htgDS_pertDay2_local
del htgDS_pertDay3_local
del htgDS_pertDay4_local
del htgDS_pertDay5_local
del htgDS_pertDay6_local


In [9]:
# del hmgDS_pertDay2
# del hmgDS_pertDay3
# del hmgDS_pertDay4
# del hmgDS_pertDay5
# del hmgDS_pertDay6

del htgDS_pertDay2
del htgDS_pertDay3
del htgDS_pertDay4
del htgDS_pertDay5
del htgDS_pertDay6


In [33]:
# ## Save upsampled (30m) datasets to pickle files for easy read-in 
# saveDir = '/glade/work/mdfowler/CLASP/histData/processedData/ens_byLeadDay/'

# pickle.dump( hmgDS_local_processed_day2,   open( saveDir+"realSfc_HMG_day2_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( hmgDS_local_processed_day3,   open( saveDir+"realSfc_HMG_day3_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( hmgDS_local_processed_day4,   open( saveDir+"realSfc_HMG_day4_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( hmgDS_local_processed_day5,   open( saveDir+"realSfc_HMG_day5_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( hmgDS_local_processed_day6,   open( saveDir+"realSfc_HMG_day6_pert00_CLUBBbudgets.p", "wb" ) )
# print('Saved processed/upsampled HMG files to pickle')

# pickle.dump( htgDS_local_processed_day2,   open( saveDir+"realSfc_HTG_day2_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day3,   open( saveDir+"realSfc_HTG_day3_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day4,   open( saveDir+"realSfc_HTG_day4_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day5,   open( saveDir+"realSfc_HTG_day5_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( htgDS_local_processed_day6,   open( saveDir+"realSfc_HTG_day6_pert00_CLUBBbudgets.p", "wb" ) )

# # pickle.dump( htg_day2_upsample,   open( saveDir+"realSfc_HTG_2018day2_30min_pert00.p", "wb" ) )
# # pickle.dump( htg_day3_upsample,   open( saveDir+"realSfc_HTGmult5_2018day3_30min_pert00.p", "wb" ) )
# # pickle.dump( htg_day4_upsample,   open( saveDir+"realSfc_HTGmult5_2018day4_30min_pert00.p", "wb" ) )
# # pickle.dump( htg_day5_upsample,   open( saveDir+"realSfc_HTGmult5_2018day5_30min_pert00.p", "wb" ) )
# # pickle.dump( htg_day6_upsample,   open( saveDir+"realSfc_HTGmult5_2018day6_30min_pert00.p", "wb" ) )
# print('Saved processed/upsampled HTG files to pickle')



Saved processed/upsampled HMG files to pickle
Saved processed/upsampled HTG files to pickle


In [5]:
# ## took a long time to get through above, so read in the processed data (not resampled)
# saveDir = '/glade/work/mdfowler/CLASP/histData/processedData/ens_byLeadDay/'

# hmgDS_local_processed_day2 = pickle.load( open( saveDir+"realSfc_HMG_day2_pert00_CLUBBbudgets.p" , "rb") )
# hmgDS_local_processed_day3 = pickle.load( open( saveDir+"realSfc_HMG_day3_pert00_CLUBBbudgets.p" , "rb") )
# hmgDS_local_processed_day4 = pickle.load( open( saveDir+"realSfc_HMG_day4_pert00_CLUBBbudgets.p" , "rb") )
# hmgDS_local_processed_day5 = pickle.load( open( saveDir+"realSfc_HMG_day5_pert00_CLUBBbudgets.p" , "rb") )
# hmgDS_local_processed_day6 = pickle.load( open( saveDir+"realSfc_HMG_day6_pert00_CLUBBbudgets.p" , "rb") )

# htgDS_local_processed_day2 = pickle.load( open( saveDir+"realSfc_HTG_day2_pert00_CLUBBbudgets.p" , "rb") )
# htgDS_local_processed_day3 = pickle.load( open( saveDir+"realSfc_HTG_day3_pert00_CLUBBbudgets.p" , "rb") )
# htgDS_local_processed_day4 = pickle.load( open( saveDir+"realSfc_HTG_day4_pert00_CLUBBbudgets.p" , "rb") )
# htgDS_local_processed_day5 = pickle.load( open( saveDir+"realSfc_HTG_day5_pert00_CLUBBbudgets.p" , "rb") )
# htgDS_local_processed_day6 = pickle.load( open( saveDir+"realSfc_HTG_day6_pert00_CLUBBbudgets.p" , "rb") )




In [10]:
## Upsample to 30m? 

# hmg_day2_upsample = hmgDS_local_processed_day2.resample(time="30Min").mean()
# hmg_day3_upsample = hmgDS_local_processed_day3.resample(time="30Min").mean()
# hmg_day4_upsample = hmgDS_local_processed_day4.resample(time="30Min").mean()
# hmg_day5_upsample = hmgDS_local_processed_day5.resample(time="30Min").mean()
# hmg_day6_upsample = hmgDS_local_processed_day6.resample(time="30Min").mean()
# print('Upsampled HMG cases (CAM)')

htg_day2_upsample = htgDS_local_processed_day2.resample(time="30Min").mean()
htg_day3_upsample = htgDS_local_processed_day3.resample(time="30Min").mean()
htg_day4_upsample = htgDS_local_processed_day4.resample(time="30Min").mean()
htg_day5_upsample = htgDS_local_processed_day5.resample(time="30Min").mean()
htg_day6_upsample = htgDS_local_processed_day6.resample(time="30Min").mean()
print('Upsampled HTG cases (CAM)')



Upsampled HTG cases (CAM)


In [11]:
## Save upsampled (30m) datasets to pickle files for easy read-in 
saveDir = '/glade/work/mdfowler/CLASP/histData/processedData/ens_byLeadDay/'

# pickle.dump( hmg_day2_upsample,   open( saveDir+"realSfc_HMG_day2_30min_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( hmg_day3_upsample,   open( saveDir+"realSfc_HMG_day3_30min_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( hmg_day4_upsample,   open( saveDir+"realSfc_HMG_day4_30min_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( hmg_day5_upsample,   open( saveDir+"realSfc_HMG_day5_30min_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( hmg_day6_upsample,   open( saveDir+"realSfc_HMG_day6_30min_pert00_CLUBBbudgets.p", "wb" ) )
# print('Saved processed/upsampled HMG files to pickle')

# pickle.dump( htg_day2_upsample,   open( saveDir+"realSfc_HTG_day2_30min_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( htg_day3_upsample,   open( saveDir+"realSfc_HTG_day3_30min_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( htg_day4_upsample,   open( saveDir+"realSfc_HTG_day4_30min_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( htg_day5_upsample,   open( saveDir+"realSfc_HTG_day5_30min_pert00_CLUBBbudgets.p", "wb" ) )
# pickle.dump( htg_day6_upsample,   open( saveDir+"realSfc_HTG_day6_30min_pert00_CLUBBbudgets.p", "wb" ) )

pickle.dump( htg_day2_upsample,   open( saveDir+"realSfc_HTGmult10_2018day2_30min_pert00_CLUBBbudgets.p", "wb" ) )
pickle.dump( htg_day3_upsample,   open( saveDir+"realSfc_HTGmult10_2018day3_30min_pert00_CLUBBbudgets.p", "wb" ) )
pickle.dump( htg_day4_upsample,   open( saveDir+"realSfc_HTGmult10_2018day4_30min_pert00_CLUBBbudgets.p", "wb" ) )
pickle.dump( htg_day5_upsample,   open( saveDir+"realSfc_HTGmult10_2018day5_30min_pert00_CLUBBbudgets.p", "wb" ) )
pickle.dump( htg_day6_upsample,   open( saveDir+"realSfc_HTGmult10_2018day6_30min_pert00_CLUBBbudgets.p", "wb" ) )
print('Saved processed/upsampled HTG files to pickle')



Saved processed/upsampled HTG files to pickle
